<a href="https://colab.research.google.com/github/sv6095/Climate-change-Faqbot/blob/main/cc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import kagglehub


path = kagglehub.dataset_download("ishgirwan/climate-change-faqs")

print("Path to dataset files:", path)


100%|██████████| 166k/166k [00:00<00:00, 27.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ishgirwan/climate-change-faqs/versions/1


In [ ]:

faq_file = os.path.join(path, "climate_change_faqs.csv")
faq_data = pd.read_csv(faq_file)



In [ ]:
print(faq_data.head())
print("Columns in the DataFrame:", faq_data.columns.tolist())

                                              source  \
0  https://www.ipcc.ch/site/assets/uploads/2020/0...   
1  https://www.ipcc.ch/site/assets/uploads/2020/0...   
2  https://www.ipcc.ch/site/assets/uploads/2020/0...   
3  https://www.ipcc.ch/site/assets/uploads/2020/0...   
4  https://www.ipcc.ch/site/assets/uploads/2020/0...   

                                                 faq text_type  
0  If Understanding of the Climate System Has Inc...         q  
1  The models used to calculate the IPCC’s temper...         a  
2               How Do We Know the World Has Warmed?         q  
3  Evidence for a warming world comes from multip...         a  
4   Have There Been Any Changes in Climate Extremes?         q  
Columns in the DataFrame: ['source', 'faq', 'text_type']


In [26]:
import pandas as pd
import os
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_md")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
questions = faq_data[faq_data['text_type'] == 'q']['faq'].tolist()
for idx, question in enumerate(questions, 1):
    print(f"{idx}. {question}")

1. If Understanding of the Climate System Has Increased, Why Hasn’t the Range of Temperature Projections Been Reduced?
2. How Do We Know the World Has Warmed?
3. Have There Been Any Changes in Climate Extremes?
4. Is the Ocean Warming?
5. Is There Evidence for Changes in the Earth’s Water Cycle?
6. How Does Anthropogenic Ocean Acidification Relate to Climate Change?
7. How Is Sea Ice Changing in the Arctic and Antarctic?
8. Are Glaciers in Mountain Regions Disappearing?
9. Is the Sun a Major Driver of Recent Changes in Climate?
10. How Unusual is the Current Sea Level Rate of Change?
11. Could Rapid Release of Methane and Carbon Dioxide from Thawing Permafrost or Ocean Warming Substantially Increase Warming?
12. What Happens to Carbon Dioxide After It Is Emitted into the Atmosphere?
13. How Do Clouds Affect Climate and Climate Change?
14. How Do Aerosols Affect Climate and Climate Change?
15. Could Geoengineering Counteract Climate Change and What Side Effects
Might Occur?
16. How Impo

In [27]:
questions = faq_data[faq_data['text_type'] == 'q']['faq'].tolist()
answers = faq_data[faq_data['text_type'] == 'a']['faq'].tolist()

fp = dict(zip(questions, answers))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    clean_text = ' '.join(tokens)
    doc = nlp(clean_text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

faq_df = pd.DataFrame(list(fp.items()), columns=['question', 'answer'])
faq_df['processed_question'] = faq_df['question'].apply(preprocess_text)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer()
faq_vectors = vectorizer.fit_transform(faq_df['processed_question'])

def gp(user_query):
    processed_query = preprocess_text(user_query)
    query_vector = vectorizer.transform([processed_query])
    tfidf_scores = cosine_similarity(query_vector, faq_vectors)[0]
    user_query_doc = nlp(processed_query)
    semantic_scores = [user_query_doc.similarity(nlp(text)) for text in faq_df['processed_question']]

    combined_scores = [0.5 * tfidf + 0.5 * semantic for tfidf, semantic in zip(tfidf_scores, semantic_scores)]
    best_idx = combined_scores.index(max(combined_scores))

    if combined_scores[best_idx] > 0.65:
        return faq_df.iloc[best_idx]['answer']
    else:
        return "I couldn't find relevant answers for your question,Try asking different question"


def chatbot():
    print("Hello! I’m Here to Answer your FAQ's. Type 'exit' to end the chat.")
    while True:
        user_query = input("You: ")
        if user_query.lower() == 'exit':
            print("Chatbot: BYe!")
            break
        response = gp(user_query)
        print("FaqBot:", response)


chatbot()

Hello! I’m Here to Answer your FAQ's. Type 'exit' to end the chat.
You: why does sun cause global warming


<ipython-input-27-aea12cfc23b8>:26: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  semantic_scores = [user_query_doc.similarity(nlp(text)) for text in faq_df['processed_question']]


FaqBot: No. The Sun can influence Earth’s climate, but it isn’t responsible for the warming trend we’ve seen over the past few decades.
You: does trees and forests play a role in this change
FaqBot: Deforestation, and especially the destruction of rainforests, is a hugely significant contributor to climate change. Scientists estimate that forest loss and other changes to the use of land account for around 23% of current man-made CO2 emissions – which equates to 17% of the 100-year warming impact of all current greenhouse-gas emissions.

As children are taught at school, trees and other plants absorb CO2 from the air as they grow. Using energy from the sun, they turn the carbon captured from the CO2 molecules into building blocks for their trunks, branches and foliage. This is all part of the carbon cycle.

A mature forest doesn't necessarily absorb much more CO2 that it releases, however, because when each tree dies and either rots down or is burned, much of its stored carbon is releas